In [1]:
from py_ecc.bn128 import G1, G2, pairing, multiply, add, neg, curve_order

In [2]:
G1

(1, 2)

In [3]:
G2

((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634),
 (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531))

In [5]:
# G2 has similar prooperties to G1

A = multiply(G2, 4)
B = multiply(G2, 8)

print("A is {}".format(A))
print("-A is {}".format(neg(A)))
print("B is {}".format(B))

assert add(A, neg(A)) is None
assert add(A, A) == B
assert multiply(A, 2) == B

A is ((18936818173480011669507163011118288089468827259971823710084038754632518263340, 18556147586753789634670778212244811446448229326945855846642767021074501673839), (18825831177813899069786213865729385895767511805925522466244528695074736584695, 13775476761357503446238925910346030822904460488609979964814810757616608848118))
-A is ((18936818173480011669507163011118288089468827259971823710084038754632518263340, 18556147586753789634670778212244811446448229326945855846642767021074501673839), (3062411694025376152460191879527889192928799351372301196444509199570489623888, 8112766110481771776007479834911244265791850668687843697874227137028617360465))
B is ((11166086885672626473267565287145132336823242144708474818695443831501089511977, 1513450333913810775282357068930057790874607011341873340507105465411024430745), (10576778712883087908382530888778326306865681986179249638025895353796469496812, 20245151454212206884108313452940569906396451322269011731680309881579291004202))


In [6]:
# Prove that products of points are equal
three = multiply(G2, 3)
four = multiply(G1, 4)

two = multiply(G2, 2)
six = multiply(G1, 6)

# Notice that the pairing funcion takes G2 first and then G1

print("Paring of three and four (G2, G1) is {}".format(pairing(three, four)))
print("Paring of two, six (G2, G1) is {}".format(pairing(two, six)))
assert pairing(three, four) == pairing(two, six)

Paring of three and four (G2, G1) is (15601246016983417620445188006067559251543215122312274362793180396629824398765, 14246743357492531742594226782128091210799645513600093008255002923531139757440, 21246528159150726701143497950911499564087497326119441605766127519735982097597, 17436280400242596046989587441318535416421721826763639552802078328724151425537, 15547233307497967574973212290527572432984848843781882171872148786972202368032, 15534527476327095590151085940887779234629190035885071294285830491019101396006, 1692483343583165135477825229458526416173742960528124726619847063046617622926, 18039014995310741533499435523514455418946747293809436454452684060284916858764, 3819483528054749127381421555378541638311106506417320721752289504154090892774, 9304846640412133065726617303096065916894576963948229503634133560057296443591, 8829429314060865060998132502145485865657576535642250650930798346925519759245, 12693463773924529243612131873073220323255622346320253723122182727824967203133)
Paring of two, six

In [7]:
# try with negative numbers
neg_two = neg(two)

print("Paring of -two, six (G2, G1) is {}".format(pairing(neg_two, six)))

Paring of -two, six (G2, G1) is (15601246016983417620445188006067559251543215122312274362793180396629824398765, 7641499514346743479652178963129183877896665643697730654434034971114086451143, 21246528159150726701143497950911499564087497326119441605766127519735982097597, 4451962471596679175256818303938739672274589330534184109886959565921074783046, 15547233307497967574973212290527572432984848843781882171872148786972202368032, 6353715395512179632095319804369495854067121121412752368403207403626124812577, 1692483343583165135477825229458526416173742960528124726619847063046617622926, 3849227876528533688746970221742819669749563863488387208236353834360309349819, 3819483528054749127381421555378541638311106506417320721752289504154090892774, 12583396231427142156519788442161209171801734193349594159054904334587929764992, 8829429314060865060998132502145485865657576535642250650930798346925519759245, 9194779097914745978634273872184054765440688810977569939566855166820259005450)


In [9]:
# Identity element of G1 and G2
# In target group, the binary operation is multiplication
Z12 = pairing(three, four) * pairing(neg_two, six)

print("Multiplication with the inverse gives the identity element {}".format(Z12))
assert pairing(three, four) * Z12 == pairing(three, four)

Multiplication with the inverse gives the identity element (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [19]:
from py_ecc.fields import optimized_bn128_FQ12 as FQ12
FQ12.one()

(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [20]:
from py_ecc.bn128 import final_exponentiate
final_exponentiate(pairing(three, four) * pairing(neg_two, six))

(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [21]:
# In solidity, the precompile 0x08 is used to verify the pairing equality
# 0 ?= -AB + CD + EF

A = multiply(G2, 6)
B = neg(multiply(G1, 2))
C = multiply(G2, 4)
D = multiply(G1, 2)
E = multiply(G2, 2)
F = multiply(G1, 2)

# It should return the identity element
final_exponentiate(pairing(A, B) * pairing(C, D) * pairing(E, F))

(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

Look at https://github.com/tornadocash/tornado-core/blob/master/contracts/Verifier.sol

In [27]:
# Homework
alpha1 = multiply(G1, 5)
beta2 = multiply(G2, 6)
gamma2 = multiply(G2, 7)
delta2 = multiply(G2, 8)

print("alpha1 is {}".format(alpha1))
print("beta2 is {}".format(beta2))
print("gamma2 is {}".format(gamma2))
print("delta2 is {}".format(delta2))

alpha1 is (10744596414106452074759370245733544594153395043370666422502510773307029471145, 848677436511517736191562425154572367705380862894644942948681172815252343932)
beta2 is ((10191129150170504690859455063377241352678147020731325090942140630855943625622, 12345624066896925082600651626583520268054356403303305150512393106955803260718), (16727484375212017249697795760885267597317766655549468217180521378213906474374, 13790151551682513054696583104432356791070435696840691503641536676885931241944))
gamma2 is ((15512671280233143720612069991584289591749188907863576513414377951116606878472, 18551411094430470096460536606940536822990217226529861227533666875800903099477), (13376798835316611669264291046140500151806347092962367781523498857425536295743, 1711576522631428957817575436337311654689480489843856945284031697403898093784))
delta2 is ((11166086885672626473267565287145132336823242144708474818695443831501089511977, 1513450333913810775282357068930057790874607011341873340507105465411024430745), (10

In [33]:
# Prover gives
A1 = multiply(G1, 53)
B2 = multiply(G2, 3)
C1 = multiply(G1, 3)
x1 = 4
x2 = 5
x3 = 6

assert final_exponentiate(pairing(B2, neg(A1)) * pairing(beta2, alpha1) * pairing(gamma2, multiply(G1, x1 + x2 + x3)) * pairing(delta2, C1)) == Z12

print("neg A1 is {}".format(neg(A1)))
print("B2 is {}".format(B2))
print("C1 is {}".format(C1))

neg A1 is (19228364643017302119251615170851391070072208144197170544602581999723466520744, 19526809854699287541348193707318917173956036703752583777232973536462226421276)
B2 is ((2725019753478801796453339367788033689375851816420509565303521482350756874229, 7273165102799931111715871471550377909735733521218303035754523677688038059653), (2512659008974376214222774206987427162027254181373325676825515531566330959255, 957874124722006818841961785324909313781880061366718538693995380805373202866))
C1 is (3353031288059533942658390886683067124040920775575537747144343083137631628272, 19321533766552368860946552437480515441416830039777911637913418824951667761761)


In [32]:
# Test pariing for two
A1 = multiply(G1, 30)
B2 = G2

assert final_exponentiate(pairing(B2, neg(A1)) * pairing(beta2, alpha1)) == Z12

print("neg A1 is {}".format(neg(A1)))
print("B2 is {}".format(B2))
print("alpha1 is {}".format(alpha1))
print("beta2 is {}".format(beta2))

neg A1 is (1527465159374431915328497116935179161014331322368960485951268517950184093102, 4614198164681446572522695455355058658980462909090786533120711656845158145809)
B2 is ((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531))
alpha1 is (10744596414106452074759370245733544594153395043370666422502510773307029471145, 848677436511517736191562425154572367705380862894644942948681172815252343932)
beta2 is ((10191129150170504690859455063377241352678147020731325090942140630855943625622, 12345624066896925082600651626583520268054356403303305150512393106955803260718), (16727484375212017249697795760885267597317766655549468217180521378213906474374, 13790151551682513054696583104432356791070435696840691503641536676885931241944))
